# Notebook to use singletCode Command Line tool to analyse single-cell data with watermelon barcodes
The barcode region is assumed to be amplified using Illumina MiSeq.

All the data for this vignette and the files output from it can be downloaded from [here](../source/dataVignette/watermelonVignetteData.zip). It contains inputFiles and the outputFiles.

### First step is to understand the samples present in the FASTQ files.
The sample fastq files are in the inputFolder. We can identify the sample name and number from the FASTQ file. For example, sampleName_S1_L001_R1_001.fastq.gz means that the sample name is sampleName and sample number is 1.Make sure that both read 1 and read 2 for each sample are present in the same folder (R1 and R2) 

### Creating sample sheet for these two samples.

In [2]:
import pandas as pd

In [5]:
Path = "path/to/where/the/repo/was/cloned"
p = "path/to/zipped/folder"

In [6]:
sampleSheet = pd.read_csv(f"{p}/inputFiles/sampleSheet.csv")
sampleSheet

,sampleName,sampleNumber
0,sampleName,1
1,otherSampleName,2


### Installing singletCode Command line tool
To use the singletCode command line tool, clone the repository from GitHub. Let the path to the folder you are running this command be **Path**

In [ ]:
!git clone https://github.com/GoyalLab/singletCodeTools

Now, to run the watermelon module of singletCodeTools, you need to run this command. If we are going by the folder structure of the zipped file and **p** is *path to the unzipped folder containing example files*, then
1. **inputFolder** will be p/inputFiles/
2. **outputFolder** will be p/outputFiles/
3. **sampleSheet** will be p/inputFiles/sampleSheet.csv

In [ ]:
import subprocess

command = [
    'python',
    f'{Path}/commandLine/singletCodeCommandLine.py',
    'watermelon',
    '-i',  f'{p}/inputFiles',
    '-o',  f'{p}/outputFiles',
    '-s', f'{p}/inputFiles/sampleSheet.csv',
    '--outputName', 'watermelonBarcodeUmi.csv'
]

result = subprocess.run(command)


**NOTE** 

In the current FASTQ file, the pattern for the watermelon barcode is GGGCTG(([AT][CG]|[CG][AT]){15})GACGCT. 
 
If this is not true for the barcodes in your data, then you can go to *processSampleBarcode* function in *Path/commandLine/watermelonUtilityFunctions.py* and change the line starting with **pattern =**.

### Using 10X list of cell IDs to check that all the cell IDs were also captured in scRNAseq

If you want to use a 10X single-cell RNA sequencing of the same set of cells to check which barcoded cells are of interest to you, then you can add --use10X flag to your command and provide the path to the list of cell IDs. 

In [ ]:
result = subprocess.run([
    'python',
    f'{Path}/commandLine/singletCodeCommandLine.py',
    'watermelon',
    '-i', f'{p}/inputFiles/',
    '-o', f'{p}/outputFiles/',
    '-s', f'{p}/inputFiles/sampleSheet.csv',
    '--outputName', 'watermelonBarcodeUmiWith10X.csv',
    '--use10X', "True",
    '--input10X', 'barcodes.tsv'
], capture_output=True, text=True)

# Check if the command was successful
if result.returncode == 0:
    print("Command executed successfully")
    print("Output:\n", result.stdout)
else:
    print("Command failed")
    print("Error:\n", result.stderr)

### Run singletCode to identify true singlets using the cellID-Barcode-UMI file just created

Using the count module available in the command line

In [ ]:
import subprocess

result = subprocess.run([
    'python',
    f'{Path}/commandLine/singletCodeCommandLine.py',
    'count',
    '-i', f'{p}/outputFiles/watermelonBarcodeUmi.csv',
    '-o', f'{p}/outputFiles/watermelon'
], capture_output=True, text=True)

# Check if the command was successful
if result.returncode == 0:
    print("Command executed successfully")
    print("Output:\n", result.stdout)
else:
    print("Command failed")
    print("Error:\n", result.stderr)

There different files which are output from this command:
1. different kinds of singlets in each of the samples: single_barcode, dominant_umi, multi_barcode
2. a combined list of all singlets for a sample: singlets_all
3. a csv file containing the statistics of each kind of singlet, number of potential multiplets and cells filtered out due to low UMI counts of barcodes 
4. the list of potential multiplets for each of the samples: multiplets

For more explanation on different kinds of singlets seen in the output files, you can refer [here](https://goyallab.github.io/SingletCodeWebsite/singletCode/) and for example of data showing this, you can refer to the vignette about singletCode package.